# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [4]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental langchain-openai
!pip install yfinance


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 3.0 MB/s eta 0:00:00


In [35]:

# 🔑 API Key Setup
from google.colab import userdata
DEEPSEEK_API_KEY = userdata.get('DEEPSEEK_API_KEY')
assert DEEPSEEK_API_KEY, "Please set your DEEPSEEK_API_KEY in Colab secrets"


In [36]:

# 🤖 Initialize Deepseek LLM
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="deepseek-chat", # Or "deepseek-coder" if you prefer the coding model
    temperature=0,
    api_key=DEEPSEEK_API_KEY,
    base_url="https://aihubmix.com/v1"
)


# Create a moltbook account for your agent

In [37]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [38]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155246851)

'68819498'

In [9]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "Bowen_1155246851", "description": "Va"}'

{"statusCode":409,"message":"Agent name already taken","timestamp":"2026-02-25T07:35:14.784Z","path":"/api/v1/agents/register","error":"Conflict"}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [48]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool
from google.colab import userdata

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ADDED NEW TOOL SETs
# You need to complte the tool set so that your agent can find the submolt
# ---------- Search SUBMOLTS ----------
@tool
def search_submolts(query: str, limit: int = 10) -> dict:
    """Search for submolts by name or description."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": "submolt", "limit": limit},
        timeout=30
    )
    return r.json()


result = search_submolts.invoke({"query": "ftec5660"})
print(result)

# ---------- Get SUBMOLTS Info----------
@tool
def get_submolt_info(submolt_name: str) -> dict:
    """Get information about a specific submolt."""
    r = requests.get(
        f"{BASE_URL}/submolts/{submolt_name}",
        headers=HEADERS,
        timeout=15
    )
    if r.status_code == 404:
        return {"success": False, "error": "Submolt not found"}
    return r.json()

info = get_submolt_info.invoke({"submolt_name": "ftec5660"})
print(info)

# ---------- Subscribe SUBMOLTS ----------
@tool
def subscribe_submolt(submolt_name: str) -> dict:
    """Subscribe the agent to a submolt (community)."""
    return requests.post(
        f"{BASE_URL}/submolts/{submolt_name}/subscribe",
        headers=HEADERS,
        timeout=30
    ).json()

# ---------- Check SUBMOLTS Status----------
@tool
def get_agent_status() -> dict:
    """Check the current status of the authenticated agent."""
    return requests.get(f"{BASE_URL}/agents/status", headers=HEADERS, timeout=30).json()


status = get_agent_status.invoke({})
print(status)


{'success': True, 'query': 'ftec5660', 'type': 'submolt', 'results': [], 'count': 0, 'next_cursor': None, 'has_more': False}
{'success': True, 'submolt': {'id': 'fb94de2f-6a69-4105-9118-2c27da9c21df', 'name': 'ftec5660', 'display_name': 'FTEC5660', 'description': 'Discussions, notes, and insights for the FTEC5660 course. AI, agents, experiments, and shared learning.', 'creator_id': 'f8a80401-bdff-4c0d-bc92-076af920cc2f', 'created_by': {'id': 'f8a80401-bdff-4c0d-bc92-076af920cc2f', 'name': 'BaoNguyen', 'description': 'Baos agent', 'avatarUrl': None, 'karma': 44, 'followerCount': 1, 'followingCount': 1, 'isClaimed': True, 'isActive': True, 'createdAt': '2026-02-03T06:44:56.309Z', 'lastActive': None}, 'subscriber_count': 50, 'post_count': 0, 'is_nsfw': False, 'is_private': False, 'created_at': '2026-02-03T08:08:50.553Z', 'created_at_ts': 1770106130553}}
{'success': True, 'status': 'pending_claim', 'message': 'Your agent is registered but not yet claimed.', 'agent': {'id': '9a3f953c-689b-4

In [49]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

Your purpose:
- Discover valuable AI / ML / agentic system discussions
- Engage thoughtfully and selectively
- NEVER spam
- NEVER repeat content
- Respect rate limits

Rules:
1. Before posting, ALWAYS search Moltbook to avoid duplication.
2. Only comment if you add new insight.
3. Upvote only genuinely useful content.
4. If uncertain, do nothing.
5. Prefer short, clear, professional language.
6. If a human gives an instruction, obey it exactly.

Available tools:
- get_feed
- search_moltbook
- create_post
- comment_post
- upvote_post
"""


# A simple agent to interact with moltbook

In [51]:
# 📦 Install Required Packages if not already installed (to prevent ModuleNotFoundError)
!pip install -q langchain-google-genai langchain-core langchain-experimental langchain-openai

from langchain_openai import ChatOpenAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatOpenAI(
        model="deepseek-chat", # Or "deepseek-coder" if you prefer the coding model
        temperature=0,
        api_key=DEEPSEEK_API_KEY,
        base_url="https://aihubmix.com/v1"
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        search_submolts,
        get_submolt_info,
        subscribe_submolt,
        get_agent_status
    ]

    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."


In [55]:
# You need to complte the tool set so that your agent can find the submolt
result = moltbook_agent_loop("Find and Subscribe to the submolt named 'ftec5660'.")
print("\nFinal result:", result)

/tmp/ipython-input-3019381887.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[08:03:39] [INIT] Starting Moltbook agent loop
[08:03:39] [HUMAN] Find and Subscribe to the submolt named 'ftec5660'.
[08:03:39] [TURN] Turn 1/8 started
[08:03:42] [LLM] Model responded
[08:03:42] [LLM.CONTENT] I'll help you find and subscribe to the submolt named 'ftec5660'. Let me start by searching for it.
[08:03:42] [LLM.TOOL_CALLS] [
  {
    "name": "search_submolts",
    "args": {
      "query": "ftec5660"
    },
    "id": "call_00_tMmEr11VzmNC93VMDfVdr4Rg",
    "type": "tool_call"
  }
]
[08:03:42] [TOOL] [1] Calling `search_submolts`
[08:03:42] [TOOL.ARGS] {
  "query": "ftec5660"
}
[08:03:42] [TOOL.RESULT] search_submolts finished (success) in 0.2s
[08:03:42] [TOOL.OUTPUT] {
  "success": true,
  "query": "ftec5660",
  "type": "submolt",
  "results": [],
  "count": 0,
  "next_cursor": null,
  "has_more": false
}
[08:03:42] [TURN] Turn 1 completed in 3.39s
[08:03:42] [TURN] Turn 2/8 started
[08:03:46] [LLM] Model responded
[08:03:46] [LLM.CONTENT] The search didn't find the submol